In [1]:
!verdi daemon restart
!reentry scan

Restarting the daemon... OK


In [2]:
import aiida
import os.path as op
from aiida import orm
from aiida.orm import load_code,Dict,StructureData,KpointsData,Int,Str,List,SinglefileData,Float
from aiida.engine import submit
from aiida_pseudo.data.pseudo.psf import PsfData

aiida.load_profile()

# Structure

In [3]:
import sisl
Si64_fdf = sisl.get_sile('/home/aakhtar/Projects/siesta/Obj_my_env/Tests/si64/si64.fdf')
Si64_geom = Si64_fdf.read_geometry()
Si64_geom_ase = Si64_geom.toASE()
Si64_geom_ase.write('host.XSF')
host = StructureData(ase=Si64_geom_ase)
iv=0 
defect = host.clone()
defect.clear_sites()
sites =host.sites
new_sites = sites
vacancy_site = new_sites.pop(iv)
[defect.append_site(s) for s in new_sites]
#len(host.sites)
#len(defect.sites)
ase_defect = defect.get_ase()
ase_defect.write('defect.XSF')

In [4]:
#from aiida_defects.formation_energy_siesta.formation_energy_siesta_manual_wk_node import FormationEnergyWorkchainSIESTAManual
from aiida_defects.formation_energy_siesta.workflows.formation_energy_siesta_neutral_manual_wk_node import FormationEnergyWorkchainSIESTAManual

Formatioan Energy Worchain for Siesta Loaded
For Charged Systems with WK Node


In [5]:
# The pseudopotentials
host_pseudos_dict = {}
#raw_pseudos = [("Si.psf", ['Si']), ("H.psf", ['H_int'])]
raw_pseudos = [("Si.psf", ['Si'])]
for fname, kinds in raw_pseudos:
    #absname = op.realpath(op.join(op.dirname(__file__), "fixtures/sample_psf", fname))
    absname = op.join("/home/aakhtar/Calculations/AiiDA/AiiDA-Defects", fname)
    pseudo = PsfData.get_or_create(absname)
    if not pseudo.is_stored:
        print("\nCreated the pseudo for {}".format(kinds))
    else:
        print("\nUsing the pseudo for {} from DB: {}".format(kinds, pseudo.pk))
    for j in kinds:
        host_pseudos_dict[j]=pseudo


Using the pseudo for ['Si'] from DB: 6009


In [6]:
# The pseudopotentials
defect_pseudos_dict = {}
#raw_pseudos = [("Si.psf", ['Si']), ("H.psf", ['H_int'])]
raw_pseudos = [("Si.psf", ['Si','Si_ghost'])]
for fname, kinds in raw_pseudos:
    #absname = op.realpath(op.join(op.dirname(__file__), "fixtures/sample_psf", fname))
    absname = op.join("/home/aakhtar/Calculations/AiiDA/AiiDA-Defects", fname)
    pseudo = PsfData.get_or_create(absname)
    if not pseudo.is_stored:
        print("\nCreated the pseudo for {}".format(kinds))
    else:
        print("\nUsing the pseudo for {} from DB: {}".format(kinds, pseudo.pk))
    for j in kinds:
        defect_pseudos_dict[j]=pseudo


Using the pseudo for ['Si', 'Si_ghost'] from DB: 6009


In [7]:
# Parameters: very coarse for speed of test
# Note the all the Si atoms are fixed...
parameters = dict={
   "mesh-cutoff": "50 Ry",
   "dm-tolerance": "0.001",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang"
    }
defect_constraints = dict={
    "%block geometryconstraints":
    """
    species-i 2 
    %endblock geometryconstraints"""
    }
relaxation = dict={
    'md-steps': 100
    }
parameters.update(relaxation)
#
# Use this for constraints
#
#parameters.update(constraints)
#
host_parameters = Dict(dict=parameters)
defect_q0_parameters = Dict(dict=parameters)
defect_q_parameters = Dict(dict=parameters)

In [8]:
host_parameters.get_dict()

{'mesh-cutoff': '50 Ry',
 'dm-tolerance': '0.001',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'md-steps': 100}

In [9]:
defect_q0_parameters.update_dict(defect_constraints)
defect_q0_parameters.get_dict()

{'mesh-cutoff': '50 Ry',
 'dm-tolerance': '0.001',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'md-steps': 100,
 '%block geometryconstraints': '\n    species-i 2 \n    %endblock geometryconstraints'}

In [10]:
defect_q_parameters.update_dict(defect_constraints)
defect_q_parameters.get_dict()

{'mesh-cutoff': '50 Ry',
 'dm-tolerance': '0.001',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'md-steps': 100,
 '%block geometryconstraints': '\n    species-i 2 \n    %endblock geometryconstraints'}

In [11]:
net_charge = dict={'NetCharge':+2.0}
VT = dict={'SaveTotalPotential': True}

In [12]:
net_charge

{'NetCharge': 2.0}

In [13]:
inputs_test = Dict(dict=defect_q_parameters.get_dict())

In [14]:
inputs_test.update_dict(net_charge)
inputs_test.update_dict(VT)

In [15]:
inputs_test.get_dict()

{'mesh-cutoff': '50 Ry',
 'dm-tolerance': '0.001',
 'DM-NumberPulay ': '3',
 'DM-History-Depth': '0',
 'SCF-Mixer-weight': '0.02',
 'SCF-Mix': 'density',
 'SCF-Mixer-kick': '35',
 'MD-VariableCell': 'F',
 'MD-MaxCGDispl': '0.3 Bohr',
 'MD-MaxForceTol': ' 0.04000 eV/Ang',
 'md-steps': 100,
 '%block geometryconstraints': '\n    species-i 2 \n    %endblock geometryconstraints',
 'NetCharge': 2.0,
 'SaveTotalPotential': True}

In [16]:
basis = Dict(
    dict={
        'pao-energy-shift':
        '300 meV',
        '%block pao-basis-sizes':
        """
        Si SZ
        %endblock pao-basis-sizes""",})

basis_defect = Dict(
    dict={
        'pao-energy-shift':
        '300 meV',
        '%block pao-basis-sizes':
        """
        Si SZ
        Si_ghost SZ
        %endblock pao-basis-sizes""",})

In [17]:
#Resources
options = Dict(dict={
    "max_wallclock_seconds": 73600,
    'withmpi': True,
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1,
    }
})


In [18]:
options.get_dict()

{'max_wallclock_seconds': 73600,
 'withmpi': True,
 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}

In [19]:
floating = {'floating_sites': [{"symbols": 'Si',
                                "name": 'Si_ghost',
                               "position": vacancy_site.position}]}

In [20]:
basis_defect.update_dict(floating)

In [21]:
basis_defect.get_dict()

{'pao-energy-shift': '300 meV',
 '%block pao-basis-sizes': '\n        Si SZ\n        Si_ghost SZ\n        %endblock pao-basis-sizes',
 'floating_sites': [{'symbols': 'Si',
   'name': 'Si_ghost',
   'position': (0.0, 0.0, 0.0)}]}

In [22]:
#codename = 'siesta-master@arsalan-pwk'
codename = 'siesta-lua@mylaptop'
code = load_code(codename)

In [23]:
#The kpoints
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1,1,1])

In [24]:
inputs = {'host_structure':host,
          'defect_structure':defect,
    'host': {'code':code,
             'parameters':host_parameters,
             'basis':basis,
             'kpoints':kpoints,
             'pseudos':host_pseudos_dict,
             'options':options},
    'defect_q0': {'code':code,
                  'parameters':defect_q0_parameters,
                  'basis':basis_defect,
                  'kpoints':kpoints,
                  'pseudos':defect_pseudos_dict,
                  'options':options},
    'defect_q': {'code':code,
                 'parameters':defect_q_parameters,
                 'basis':basis_defect,
                 'kpoints':kpoints,
                 'pseudos':defect_pseudos_dict,
                 'options':options},
    'defect_charge':Float(-1),
    'defect_site':List(list=[0.0,0.0,0.0]),
    'chemical_potential':Float(-100.629519),
    'epsilon':Float(9.0),
    'wk_node': Int(9282),#Int(9093),
    'add_or_remove':Str('remove'),
    #'correction_scheme':Str('rho')     
}



In [25]:
inputs

{'host_structure': <StructureData: uuid: 20699b02-3ada-42ad-94a5-ec1690d7c06b (unstored)>,
 'defect_structure': <StructureData: uuid: c1f127f0-1767-45b8-a003-5eab471ebc4a (unstored)>,
 'host': {'code': <Code: Remote code 'siesta-lua' on mylaptop, pk: 1, uuid: f044d7db-fff3-48dd-b489-6f81eb736d8d>,
  'parameters': <Dict: uuid: 0296a6b1-39fd-4c7e-bf08-5e9b92aaf1d6 (unstored)>,
  'basis': <Dict: uuid: ee808f34-d6e0-4268-8881-2237758d1713 (unstored)>,
  'kpoints': <KpointsData: uuid: 29913db9-983e-40ec-a1df-b4e2fa75cc88 (unstored)>,
  'pseudos': {'Si': <PsfData: uuid: 1eb6fc02-9076-4476-8cd2-6a3e73bacaa3 (pk: 6009)>},
  'options': <Dict: uuid: 4b36585c-1970-44ae-ac72-38d019ce7380 (unstored)>},
 'defect_q0': {'code': <Code: Remote code 'siesta-lua' on mylaptop, pk: 1, uuid: f044d7db-fff3-48dd-b489-6f81eb736d8d>,
  'parameters': <Dict: uuid: 2251b53a-8dbd-4b44-b370-64ee244f3d98 (unstored)>,
  'basis': <Dict: uuid: e3a52d72-71d7-48cd-873d-b7b8af37592e (unstored)>,
  'kpoints': <KpointsData: u

In [26]:
process = submit(FormationEnergyWorkchainSIESTAManual, **inputs)
print("Submitted VacancyExchangeBarrier workchain; ID={}".format(process.pk))
print(
    "For information about this workchain type: verdi process show {}".format(
        process.pk))
print("For a list of running processes type: verdi process list")


Submitted VacancyExchangeBarrier workchain; ID=9518
For information about this workchain type: verdi process show 9518
For a list of running processes type: verdi process list


In [27]:
#node =orm.load_node(9398)

In [28]:
#node.outputs.defect_q0_structure